# Sinusfunktion mit neuronalem Netz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

!python --version
print("NumPy: " + np.__version__)
print("Keras: " + tf.keras.__version__)
print("TensorFlow: " + tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
nsamples   = 1000    # Anzahl Stichproben
val_ratio  = 0.2     # Prozentsatz der Stichprobe für Validierung
test_ratio = 0.2     # Prozentsatz der Stichprobe für Test

# Erzeugung einer zufälligen Stichprobe
np.random.seed(1234)
x_values = np.random.uniform(low=0, high=(2 * math.pi), size=nsamples)
plt.plot(x_values, '.')

In [ ]:
# anhand Stichprobe werden verrauschte Sinuswerte berechnet
y_values = np.sin(x_values) + (0.1 * np.random.randn(x_values.shape[0]))
plt.plot(x_values, y_values, '.')

In [ ]:
# Aufteilung in Trainings-, Validierungs- und Testdatensatz
val_split  = int(val_ratio * nsamples)
test_split = int(val_split + (test_ratio * nsamples))

x_val, x_test, x_train = np.split(x_values, [val_split, test_split])
y_val, y_test, y_train = np.split(y_values, [val_split, test_split])

# Test, ob Aufteilung der Samples korrekt ist
assert(x_train.size + x_val.size + x_test.size) == nsamples

In [ ]:
# Plot the data in each partition in different colors:
plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.plot(x_val, y_val, 'y.', label="Validate")
plt.legend()
plt.show()

In [ ]:
# Modellentwurf
model = tf.keras.Sequential()

model.add(tf.keras.layers.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

In [ ]:
# Optimierer, Verlustfunktion und Metriken
model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

In [ ]:
# Training des Modells
history = model.fit(x_train, y_train, epochs=500, batch_size=100, verbose=2, validation_data=(x_val, y_val))

In [ ]:
# Trainingsverlauf grafisch darstellen
epochs = range(1, len(history.history['val_loss']) + 1)

plt.figure(figsize=(8,7))
plt.plot(epochs, history.history['val_loss'])
plt.xlabel('Epoche')
plt.ylabel('Fehler')
plt.grid(True)

In [ ]:
# Vergleich zwischen Vorhersagen und tatsächliche Werten
predictions = model.predict(x_test)

plt.clf()
plt.title("Vergleich der Ergebnisse")
plt.plot(x_test, y_test, 'b.', label='Actual')
plt.plot(x_test, predictions, 'r.', label='Prediction')
plt.legend()

In [ ]:
# Speichern des Keras-Models
KERAS_MODEL_FILENAME = 'sine_model.h5'
model.save(KERAS_MODEL_FILENAME)

### 2. Konvertierung für TFLite-Micro

In [ ]:
# Quantisierung und Optimierung für Microcontroller

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimazations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

TFLITE_MODEL_FILENAME = "sine_model.tflite"
open(TFLITE_MODEL_FILENAME, "wb").write(tflite_model);

### 2.1 Manuelle Konvertierung des tflite-Modells in Header-Datei

In [ ]:
!echo "const unsigned char model[] = {"  > sine_model.h
!cat sine_model.tflite | xxd -i         >> sine_model.h
!echo "};"                              >> sine_model.h